In [ ]:
! pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=490e0ab0138ed3e12994b0994a772d599040c6b80779adc2c1792f14b461f8a3
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
import pandas as pd 
import numpy as np
from pathlib import Path
import hashlib
from functools import partial
import os

In [ ]:
wget.download("https://www.openslr.org/resources/12/md5sum.txt", "", wget.bar_thermometer)

[]

'md5sum.txt'

In [ ]:
wget.download("https://www.openslr.org/resources/12/dev-clean.tar.gz", "", wget.bar_thermometer)
wget.download("https://www.openslr.org/resources/12/dev-other.tar.gz", "", wget.bar_thermometer)


[]

'dev-other.tar.gz'

In [ ]:
src = "/content"
def load_md5sum_file(src):
  ds = pd.read_csv(src+"/md5sum.txt",names=['md5sum', 'split'], sep=" ")
  ds["md5sum"] = ds.index
  ds.set_index(pd.RangeIndex(start=0, stop=ds.shape[0]))

  return ds

ds = load_md5sum_file(src)

In [ ]:
def md5sum(file, bufsize=1<<15):
    d = hashlib.md5()
    for buf in iter(partial(file.read, bufsize), b''):
        d.update(buf)
    return d.hexdigest()

def calculate_md5sum(main, pattern = '*.tar.gz'):
  names = []
  md    = []
  for src in Path(main).rglob(pattern):
    f = open(src, "rb")
    name =  str(src).split("/")[-1]
    names.append(name)
    md.append(str(md5sum(f)))

  df = {
      "split" :names,
      "md5sum":md
  }
  return pd.DataFrame(df)

ds2 = calculate_md5sum(main="/content/", pattern = '*.tar.gz')

In [ ]:
ds3 = ds.merge(ds2, on=["split"], how="inner")


match     = ds3[ds3.md5sum_x == ds3.md5sum_y].split.tolist()
not_match = ds3[ds3.md5sum_x != ds3.md5sum_y].split.tolist()

missing   = list(set(ds.split.tolist()).difference(set(ds2.split.tolist())))

In [ ]:
def check_librispeech_md5sum(src):
  os.remove(src+"/md5sum.txt")
  wget.download("https://www.openslr.org/resources/12/md5sum.txt", src)
  
  real_sum         = load_md5sum_file(src)
  calculated_sum   = calculate_md5sum(main="/content/", pattern = '*.tar.gz') #update this line 
  sum_intersection = real_sum.merge(calculated_sum, on=["split"], how="inner")


  match     = sum_intersection[sum_intersection.md5sum_x ==
                               sum_intersection.md5sum_y].split.tolist()

  not_match = sum_intersection[sum_intersection.md5sum_x !=
                               sum_intersection.md5sum_y].split.tolist()

  missing   = list(set(ds.split.tolist()).difference(set(ds2.split.tolist())))

  return match, not_match, missing

In [ ]:
def clean_split_name(splits):
  return [x.split(".")[0] for x in splits]

In [ ]:
def what_to_download(src, splits):
  match, not_match, missing = check_librispeech_md5sum(src)

  match     = clean_split_name(match)
  not_match = clean_split_name(not_match)
  missing   = clean_split_name(missing)

  for i in not_match:
    os.remove(src+"/d/"+i+".tar.gz")
    
  missing.extend(not_match)
  required = missing
  download = list(set(splits).intersection(set(required)))

  return download

In [ ]:
what_to_download(src=".", splits=[
                                 "train-other-500",
                                 "train-clean-360",
                                 "dev-clean",
                                  ])

['train-clean-360', 'train-other-500']

In [ ]:
print("found %d missing splits, " %(len(missing)), missing)
print("found %d match splits, " %(len(match)), match)
print("found %d not match splits, " %(len(not_match)), not_match)


found 9 missing splits,  ['train-other-500.tar.gz', 'train-clean-360.tar.gz', 'original-mp3.tar.gz', 'intro-disclaimers.tar.gz', 'train-clean-100.tar.gz', 'raw-metadata.tar.gz', 'original-books.tar.gz', 'test-other.tar.gz', 'test-clean.tar.gz']
found 2 match splits,  ['dev-clean.tar.gz', 'dev-other.tar.gz']
found 0 not match splits,  []


In [ ]:
type(ds2.split[0])


str

In [ ]:
type(ds.md5sum[0])


str

In [ ]:
type(ds2.md5sum[0])


str

In [ ]:
ds2.index

RangeIndex(start=0, stop=2, step=1)

In [ ]:
ds.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64')